# Adatom Defect in Graphene

## 0. Introduction

This tutorial demonstrates creation of metal adatom on graphene hollow site.

> **Chan, K. T., Neaton, J. B., & Cohen, M. L. (2008). 
> First-principles study of metal adatom adsorption on graphene. Physical Review B, 77(23). 
> [DOI](doi:10.1103/physrevb.77.235430 )


Below is the figure 1 from the manuscript that shows the adsorption of metal adatom on graphene.


![Fig 1.](https://i.imgur.com/adatom_graphene.png)



## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L102).

Note that we use approximate coordinates in crystal coordinates to define the defect positions. The approximate coordinates are used to find the closest site in the crystal to the given coordinates. The approximate coordinates are in the range of [0, 1] for each axis. Adjust the approximate coordinates to place the defect at the desired site in the crystal. Using the visuals provided in the notebook could help in determining the approximate coordinates.
(Coordinates found using 3D editor in Materials Designer)

In [ ]:
# Element to be placed at the site (ignored for vacancy)
MATERIAL_NAME = "Graphene"

SUPERCELL_MATRIX = [[4, 0, 0], [0, 4, 0], [0, 0, 1]]
USE_CARTESIAN_COORDINATES = False

# Defect parameters
DEFECT_TYPE = "adatom"
PLACEMENT_METHOD = "equidistant"
DISTANCE_Z = 2.5
APPROXIMATE_POSITION_ON_SURFACE = [0.5, 0.5]
CHEMICAL_ELEMENT = "Ti"


### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("create_point_defect.ipynb", "../../../config.yml")

### 1.3. Load input material
In this notebook we will use the material from the `uploads` folder that has a few pre-set materials.

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

material = Material(Materials.get_by_name_first_match(MATERIAL_NAME))

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

supercell = create_supercell(material, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the Defect
### 2.1. Initialize Configuration and Builder parameters

In [ ]:
from mat3ra.made.tools.build.defect import  AdatomSlabPointDefectConfiguration
from mat3ra.made.tools.build.defect.builders import PointDefectBuilderParameters

adatom_config = AdatomSlabPointDefectConfiguration(crystal=supercell, 
                                              defect_type=DEFECT_TYPE, 
                                              placement_method=PLACEMENT_METHOD,
                                              chemical_element=CHEMICAL_ELEMENT, 
                                              distance_z=DISTANCE_Z, 
                                              position_on_surface=APPROXIMATE_POSITION_ON_SURFACE
)

defect_builder_parameters = PointDefectBuilderParameters()

### 2.2. Create the defects

In [ ]:
from mat3ra.made.tools.build.defect import create_slab_defect

material_with_defect = create_slab_defect(adatom_config)

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}],
          rotation="-90x")
visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}])

## 4. Write resulting material to the folder

In [ ]:
from utils.jupyterlite import download_content_to_file
material_with_defect.name = "N-doped Graphene"
download_content_to_file(material_with_defect.to_json(), "N-doped_Graphene.json")